In [15]:
import json
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import ast

emb_model = SentenceTransformer('moka-ai/m3e-base')

data_dict_df = pd.read_csv('data/data_dictionary.csv')
db_description_json = json.load(open('data/database-table/database-with_description.json'))

table_info_dict = {}
for db in db_description_json:
    key = db['table_name_en']
    value = {
        'database_name_zh': db['database_name_zh'],
        'database_name_en': db['database_name_en'],
        'table_name_en': db['table_name_en'],
        'table_name_zh': db['table_name_zh'],
        'table_description' : db['table_description'],
    }
    table_info_dict[key] = value 


from tqdm import tqdm


columns_info = []

for i, row in tqdm(data_dict_df.iterrows(), total=data_dict_df.shape[0]):
    table_name_en = row['table_name']
    column_name_en = row['column_name']
    db_name = table_info_dict[table_name_en]['database_name_en']
    column_description = (row['column_description'] if isinstance(row['column_description'], str) else '')
    annotation = (row['注释'] if isinstance(row['注释'], str) else '')
    table_description = table_info_dict[table_name_en]['table_description']
    table_name_zh = table_info_dict[table_name_en]['table_name_zh']

    columns_info.append({
        'db_name': table_info_dict[table_name_en]['database_name_en'],
        'table_name_en': table_name_en,
        'column_name': column_name_en,
        'column_description': column_description,
        'table_name_zh': table_name_zh,
        'table_name_zh_emb': emb_model.encode(table_name_zh),
        'table_desc_emb': emb_model.encode(table_description),
        'col_emb': emb_model.encode(column_description + ' ' + annotation),
        'all_emb': emb_model.encode(table_name_zh + ': ' + table_description + ' ' + column_description + ' ' + annotation),
    })


100%|██████████| 3489/3489 [05:03<00:00, 11.49it/s]


保存embedding为文件

In [16]:
def convert_embeddings(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, dict):
        return {key: convert_embeddings(value) for key, value in obj.items()}
    if isinstance(obj, list):
        return [convert_embeddings(item) for item in obj]
    return obj

# Convert dictionaries with embeddings to JSON serializable format
columns_info_serializable = convert_embeddings(columns_info)

with open('data/database-table/columns_emb.json', 'w', encoding='utf-8') as f:
    json.dump(columns_info_serializable, f, ensure_ascii=False, indent=4)

    

读取保存的文件

In [1]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer('moka-ai/m3e-base')

def parse_embedding(embedding_str):
    """
    将存储在 JSON 中的嵌入字符串转换为 numpy 数组。
    假设嵌入存储为形如 "[0.1, 0.2, ...]" 的字符串。
    """
    return np.array(ast.literal_eval(embedding_str))

# 读取 JSON 文件并转换回字典
def load_json_to_dict(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 递归解析嵌入数据
    def convert_embeddings(obj):
        if isinstance(obj, list) and all(isinstance(x, (int, float)) for x in obj):
            return np.array(obj)
        if isinstance(obj, dict):
            return {key: convert_embeddings(value) for key, value in obj.items()}
        if isinstance(obj, list):
            return [convert_embeddings(item) for item in obj]
        return obj
    
    return convert_embeddings(data)

# 加载 JSON 文件到字典
columns_info = load_json_to_dict('data/database-table/columns_emb.json')

以下计算：
score = 0.5 * table_description_cos + 0.25 * (column_name_cos + column_annotation_cos)

In [2]:
import sys
import os
import json
import time
import re

from tqdm import tqdm

cwd = os.getcwd()
os.chdir(cwd)
sys.path.append('tools')

import chat
import parse_data
import sql

question_path = os.path.join(cwd, 'answer_tmp' + os.sep + 'glm_4_plus-market_classifier-v1.0.0.json')

questions = parse_data.read_json(question_path)
# sort the questions by tid
questions = sorted(questions, key=lambda x: int(x['tid'].split('-')[-1]))

In [9]:
import numpy as np
from sentence_transformers import util

def cosine_similarity(vec_a, vec_b):
    return np.dot(vec_a, vec_b) / (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))

def find_top_similar_columns(word, columns_info, emb_model, top_k=20):
    """
    Find the top `top_k` most similar columns to the given word based on cosine similarity.

    Args:
    - word (str): The query word.
    - columns_info (list): list containing column details with embeddings.
    - emb_model (SentenceTransformer): The embedding model.
    - top_k (int): Number of top similar columns to return.

    Returns:
    - List of tuples (column_name, similarity_score, column_description)
    """
    # Encode the input word
    word_embedding = emb_model.encode(word)

    table_similarities = []
    column_similarities = []
    scores1 = []
    scores2 = []

    for col_info in columns_info:
        # Get column name and description
        column_name = col_info['column_name']
        column_description = col_info.get('column_description', '')
        db_name = col_info.get('db_name', '')
        table_name_en = col_info.get('table_name_en', '')
        table_name_zh = col_info.get('table_name_zh', '')


        # Get embeddings (ensure they exist)
        table_name_zh_emb = col_info.get('table_name_zh_emb')
        table_desc_emb = col_info.get('table_desc_emb')
        col_emb = col_info.get('col_emb')
        all_emb = col_info.get('all_emb')

        table_name_similarity = cosine_similarity(word_embedding, table_name_zh_emb) if table_name_zh_emb is not None else 0
        table_desc_similarity = cosine_similarity(word_embedding, table_desc_emb) if table_desc_emb is not None else 0
        col_similarity = cosine_similarity(word_embedding, col_emb) if col_emb is not None else 0
        all_similarity = cosine_similarity(word_embedding, all_emb) if all_emb is not None else 0

        # 四种similarity的计算方法

        table_similarity = 0.5 * table_name_similarity + 1 * table_desc_similarity 
        if table_name_en not in [x[2] for x in table_similarities]:
            table_similarities.append((table_similarity, db_name, table_name_en, table_name_zh))

        column_similarities.append((col_similarity, db_name, table_name_en, column_name, column_description))

        score1 = all_similarity
        scores1.append((score1, db_name, table_name_en, column_name, column_description))

        score2 = 0.3 * table_similarity + 0.7 * col_similarity
        scores2.append((score2, db_name, table_name_en, column_name, column_description))


    # Sort by similarity score
    table_similarities = sorted(table_similarities, key=lambda x: x[0], reverse=True)
    column_similarities = sorted(column_similarities, key=lambda x: x[0], reverse=True)
    scores1 = sorted(scores1, key=lambda x: x[0], reverse=True)
    scores2 = sorted(scores2, key=lambda x: x[0], reverse=True)


    # Return top `top_k` results
    return table_similarities[:top_k], column_similarities[:top_k], scores1[:top_k], scores2[:top_k]

entities = ['卧龙电气驱动集团股份有限公司2019年年度报告中，未调整的合并资产负债表中提到的资产总计是多少？',
            '大北农在2020年发布了多少条重大事项公告？',
            '卧龙电气驱动集团股份有限公司的注册地在哪个省份？',
            '湖北济川药业股份有限公司上市以来十大股东的类型有哪些？',
            '东山精密最近一期员工持股计划的参与总人数是多少？其中管理层参与人数占比(四舍五入精确到小数点后两位，并以百分比形式表示)是多少？',
            '最新更新的2020半年报中，机构持有无限售流通A股数量合计最多的公司简称是？',
            '新科技纳入过多少个子类概念？', '中文全称', '全称变更', 'A股简称','法人','法律顾问','会计师事务所','董秘', '实控人', '近一个月最高价', '现金流量净额',
            '注册邮箱', '注册地址', '信披网址', '公司电话','硕士及以上学历（硕士+博士）的人员占比', '一级行业', '收盘价',
            '什么时间上市']

# entities = [i['team'][0]['question'] for i in questions][2:3]

available_tables = set()

for entity in entities:
    print(f"{entity}: ")
    results = find_top_similar_columns(entity, columns_info, emb_model, top_k=10)
    print("- By Table Only:")
    for result in results[0]:
        print(f'  - {result[1]}.{result[2]} ({result[3]})')
        available_tables.add(f'{result[1]}.{result[2]}')
    print("- By Column Only:")
    for result in results[1]:
        print(f'  - {result[1]}.{result[2]}.{result[3]} ({result[4]})')
        available_tables.add(f'{result[1]}.{result[2]}')
    print("- By Total Scores 1")
    for result in results[2]:
        print(f'  - {result[1]}.{result[2]}.{result[3]} ({result[4]})')
        available_tables.add(f'{result[1]}.{result[2]}')
    print("- By Total Scores 2")
    for result in results[2]:
        print(f'  - {result[1]}.{result[2]}.{result[3]} ({result[4]})')
        available_tables.add(f'{result[1]}.{result[2]}')

    print()

卧龙电气驱动集团股份有限公司2019年年度报告中，未调整的合并资产负债表中提到的资产总计是多少？: 
- By Table Only:
  - AStockEventsDB.LC_Regroup (公司资产重组明细)
  - AStockFinanceDB.LC_BalanceSheetAll (资产负债表_新会计准则)
  - AStockFinanceDB.LC_CapitalInvest (资金投向说明)
  - AStockFinanceDB.LC_CashFlowStatementAll (现金流量表_新会计准则)
  - AStockFinanceDB.LC_IncomeStatementAll (利润分配表_新会计准则)
  - AStockShareholderDB.LC_ShareStru (公司股本结构变动)
  - AStockFinanceDB.LC_MainOperIncome (公司主营业务构成)
  - AStockShareholderDB.LC_ShareTransfer (股东股权变动)
  - AStockFinanceDB.LC_IntAssetsDetail (公司研发投入与产出)
  - AStockEventsDB.LC_MajorContract (公司重大经营合同明细)
- By Column Only:
  - AStockFinanceDB.LC_IncomeStatementAll.AssetImpairmentLoss (资产减值损失)
  - AStockFinanceDB.LC_BalanceSheetAll.TConstruInProcess (在建工程合计)
  - AStockFinanceDB.LC_BalanceSheetAll.TotalFixedAsset (固定资产合计1)
  - AStockFinanceDB.LC_BalanceSheetAll.TotalAssets (资产总计(元))
  - AStockFinanceDB.LC_BalanceSheetAll.TotalLiabilityAndEquity (负债及股东权益总计(元))
  - AStockFinanceDB.LC_BalanceSheetAll.TotalCurrentLiability (流动负债合计(元))

In [98]:
len(available_tables)

10

In [99]:
import pandas as pd

table_fpath = 'data/database-table/database_v4.md'

df = pd.read_table(table_fpath, sep="|", skiprows=[1], engine='python')
df = df.iloc[:, 1:-1]  # 去掉多余的边界列
df.columns = [col.strip() for col in df.columns]  # 去掉列名的空格
df

,表英文,表中文,表描述,数据范围,信息来源
0,ConstantDB.HK_SecuMain,港股证券主表,记录港股单个证券品种的简称、中英文名、上市交易、上市状态所等基础信息。,NaN,
1,ConstantDB.US_SecuMain,美股证券主表,记录美国等境外市场单个证券品种的简称、中英文名、上市交易所、上市状态等基础信息。,NaN,NaN
2,ConstantDB.SecuMain,证券主表,记录A股单个证券品种（股票、基金、债券）的代码、简称、中英文名、上市交易所、上市板块、上市...,NaN,NaN
3,ConstantDB.CT_SystemConst,系统常量表,本表收录数据库中各种常量值的具体分类和常量名称描述。,NaN,NaN
4,ConstantDB.LC_AreaCode,国家城市代码表,本表收录世界所有国家层面的数据信息和我国不同层级行政区域的划分信息。,NaN,NaN
...,...,...,...,...,...
72,IndexDB.LC_IndexBasicInfo,指数基本情况,收录了市场上主要指数的基本情况，包括指数类别、成份证券类别、发布机构、发布日期、基期基点、...,NaN,中证指数有限公司、上海证券交易所、深圳证券交易所、中央国债登记结算有限责任公司、申银万国研...
73,IndexDB.LC_IndexComponent,指数成份,1. 收录了市场上主要指数的成份证券构成情况，包括成份证券的市场代码、入选日期、删除日期以及...,1990-12 ~ 至今,中证指数有限公司、上海证券交易所、深圳证券交易所、申银万国研究所等
74,InstitutionDB.PS_EventStru,事件体系指引表,收录聚源最新制定的事件分类体系。,NaN,NaN
75,InstitutionDB.LC_InstiArchive,机构基本资料,收录市场上重要机构的基本资料情况，如证券公司、信托公司、保险公司等；包含机构名称、机构信息...,NaN,国家企业信用信息公示系统等


In [100]:
new_df = df[:6].copy()

for index, row in df.iterrows():
    rname = row['表英文']
    rname= re.sub('\s', '', rname)
    if rname in available_tables:
        new_df = pd.concat([new_df, pd.DataFrame([row])], ignore_index=True)

new_df

,表英文,表中文,表描述,数据范围,信息来源
0,ConstantDB.HK_SecuMain,港股证券主表,记录港股单个证券品种的简称、中英文名、上市交易、上市状态所等基础信息。,NaN,
1,ConstantDB.US_SecuMain,美股证券主表,记录美国等境外市场单个证券品种的简称、中英文名、上市交易所、上市状态等基础信息。,NaN,NaN
2,ConstantDB.SecuMain,证券主表,记录A股单个证券品种（股票、基金、债券）的代码、简称、中英文名、上市交易所、上市板块、上市...,NaN,NaN
3,ConstantDB.CT_SystemConst,系统常量表,本表收录数据库中各种常量值的具体分类和常量名称描述。,NaN,NaN
4,ConstantDB.LC_AreaCode,国家城市代码表,本表收录世界所有国家层面的数据信息和我国不同层级行政区域的划分信息。,NaN,NaN
5,ConstantDB.QT_TradingDayNew,交易日表(新),本表收录各个市场的交易日信息，包括给定日期是否是交易日，是否周、月、季、年最后一个交易日。,NaN,NaN
6,AStockOperationsDB.LC_RewardStat,公司管理层报酬统计,按报告期统计管理层的报酬情况，包括报酬总额、前三名董事报酬、前三名高管报酬、报酬区间统计分析等。,2001-12-31 ~ 至今,定期报告、招股说明书等
7,AStockShareholderDB.LC_SHNumber,股东户数,1. 反映公司全体股东、A股股东、B股东、H股东、CDR股东的持股情况及其历史变动情况等。...,1991年 ~ 至今,招股说明书、上市公告书、定报、临时公告、深交所互动易、上证e互动等
8,AStockShareholderDB.LC_ESOP,员工持股计划,主要记录员工持股计划当期的情况：包括相关日期、事件进程、事件说明、资金来源、资金总额、股票...,2014-06 ~ 至今,上市公司公告
9,AStockShareholderDB.LC_ESOPSummary,员工持股计划概况,1.本表主要记录员工持股计划总体情况：包括相关日期、事件进程、事件说明、资金来源、资金总额、...,2014-06 ~ 至今,上市公司公告


In [109]:
# 转换成 Markdown 格式
markdown_table = new_df.to_markdown(index=False)

# 去除多余的空格和横线
markdown_table = '\n'.join(re.sub('  ', '', line) for line in markdown_table.splitlines() if line.strip())
# 去除多余的 --
markdown_table = re.sub('\|:-+\|:-+\|:-+\|:-+\|:-+\|', '|---|---|---|---|---|', markdown_table)

print(markdown_table)

| 表英文| 表中文| 表描述 | 数据范围| 信息来源|
|---|---|---|---|---|
| ConstantDB.HK_SecuMain| 港股证券主表| 记录港股单个证券品种的简称、中英文名、上市交易、上市状态所等基础信息。 | nan | |
| ConstantDB.US_SecuMain| 美股证券主表| 记录美国等境外市场单个证券品种的简称、中英文名、上市交易所、上市状态等基础信息。 | nan | nan |
| ConstantDB.SecuMain | 证券主表| 记录A股单个证券品种（股票、基金、债券）的代码、简称、中英文名、上市交易所、上市板块、上市状态等基础信息。| nan | nan |
| ConstantDB.CT_SystemConst | 系统常量表| 本表收录数据库中各种常量值的具体分类和常量名称描述。 | nan | nan |
| ConstantDB.LC_AreaCode| 国家城市代码表| 本表收录世界所有国家层面的数据信息和我国不同层级行政区域的划分信息。 | nan | nan |
| ConstantDB.QT_TradingDayNew | 交易日表(新)| 本表收录各个市场的交易日信息，包括给定日期是否是交易日，是否周、月、季、年最后一个交易日。 | nan | nan |
| AStockOperationsDB.LC_RewardStat| 公司管理层报酬统计| 按报告期统计管理层的报酬情况，包括报酬总额、前三名董事报酬、前三名高管报酬、报酬区间统计分析等。 | 2001-12-31 ~ 至今 | 定期报告、招股说明书等|
| AStockShareholderDB.LC_SHNumber | 股东户数| 1. 反映公司全体股东、A股股东、B股东、H股东、CDR股东的持股情况及其历史变动情况等。<br>2.指标计算公式：<br>\t1)户均持股比例＝((股本/股东总户数)/股本)*100%（公式中分子分母描述同一股票类型）。<br>\t2)相对上一期报告期户均持股比例变化＝本报告期户均持股比例-上一报告期户均持股比例。<br>\t3)户均持股数季度增长率＝(本季度户均持股数量/上一季度户均持股数量-1)*100%。<br>\t4)户均持股比例季度增长率=(本季度户均持股比例/上一季度户均持股比